## Disclaimer: this was not used in the end

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.utils import class_weight

Plan:
- data analysis for each dataset - done in other files
- merge them
- take out around 10 5 datapoints for validation later
- apply algorithm to label all data

- export so i can use it on DL algo


https://www.kaggle.com/code/adityarahul/fakenewsdetection

load all labelled data and merge it

In [2]:
def merge_csv_files(folder_path):
    all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
    dataframes = [pd.read_csv(file) for file in all_files]
    merged_df = pd.concat(dataframes, ignore_index=True)
    return merged_df

In [3]:
labelled_folder = r"C:\github\news\news\german-news\preprocess data\data_ready_for_analysis\labelled"

In [4]:
labelled_data = merge_csv_files(labelled_folder)

In [5]:
labelled_data

,news_outlet,provenance,query_keywords,creation_date,last_modified,crawl_date,author_person,author_organization,news_keywords,title,description,body,Fake News,Extreme bias,clickbait,credible,body_len,has_label,label
0,bild,https://www.bild.de/politik/ausland/politik-au...,"linke, migration, csu, cdu",2022-06-15,2022-06-15,2024-10-20,Filipp Piatov und Ralf Schuler,NaN,NaN,Sie verspottete Deutsche als Kartoffeln: Aktiv...,Schon wieder sorgt eine Personalentscheidung d...,Schon wieder sorgt eine Personalentscheidung d...,1.0,1.0,1.0,0.0,3574.0,True,"[1, 1, 1, 0]"
1,bild,https://www.bild.de/politik/ausland/politik-au...,israel,2023-11-10,2023-11-10,2024-10-20,"P. Tiede, A. Link und H.-J. Vehlewald",NaN,NaN,Reporter als Terror-Helfer: Die perfide Propag...,"Es ist der 7. Oktober 2023. Der Tag, der alles...","Es ist der 7. Oktober 2023. Der Tag, der alles...",1.0,1.0,0.0,0.0,5226.0,True,"[1, 1, 0, 0]"
2,bild,https://www.bild.de/politik/inland/politik-inl...,"fdp, csu, spd, cdu, afd, annalena baerbock, ch...",2023-06-19,2023-06-19,2024-10-20,Filipp Piatov,NaN,"CDU, CSU, die Gruenen, SPD, Wuest Hendrik, Mer...",Streit in der CDU: Umfrage-Schlacht zwischen M...,In der CDU liegen die Nerven blank! Es tobt ei...,In der CDU liegen die Nerven blank! Spaetesten...,0.0,1.0,0.0,0.0,3071.0,True,"[0, 1, 0, 0]"
3,bild,https://www.bild.de/politik/ausland/politik-au...,"israel, spd, olaf scholz",2023-10-30,2023-10-30,2024-10-20,NaN,BILD,"Gaza-Streifen, Gaza, Israel, Hamas, Nahost-Kon...",Shani Louk ermordet Scholz: Zeigt die ganze B...,Shani Louk wurde barbarisch ermordet. Das best...,Es ist das pure Grauen. Am 7. Oktober verschle...,0.0,0.0,0.0,1.0,2655.0,True,"[0, 0, 0, 1]"
4,bild,https://www.bild.de/regional/leipzig/leipzig-n...,"abschiebung, migration, cdu",2023-11-09,2023-11-09,2024-10-20,KARL KEIM,NaN,"Tunesien, Georgien, Sachsen, asylrecht, Schust...",Sachsen: Nur jede dritte Abschiebung findet wi...,In Sachsen klappt nur jede dritte Abschiebung....,Leipzig In Sachsen wurden bisher in diesem Ja...,0.0,0.0,0.0,1.0,2447.0,True,"[0, 0, 0, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,welt,https://www.welt.de/politik/ausland/article242...,"krieg, christ, ss, sp, ukraine, ns, spo, nazi,...",2022-12-12,2022-12-12,2024-10-20,NaN,NaN,"Newsteam, Russland-Ukraine-Krieg (24.2.2022), ...",Ukraine-News ++ Ukraines Verteidigungsminister...,Laut dem ukrainischen Verteidigungsminister bi...,Laut dem ukrainischen Verteidigungsminister bi...,0.0,0.0,0.0,1.0,21564.0,True,"[0, 0, 0, 1]"
337,welt,https://www.welt.de/politik/deutschland/articl...,"sp, terror, ns, spd, schi, rechtsextremismus, ss",2022-12-13,2022-12-13,2024-10-20,NaN,NaN,"Newsteam, Wehrbeauftragte, Hoegl, Eva, Reichsb...",Wehrbeauftragte fordert haerteres Vorgehen geg...,Nach der Razzia gegen ein mutmassliches Terror...,Nach der Razzia gegen ein mutmassliches Terror...,0.0,0.0,0.0,1.0,972.0,True,"[0, 0, 0, 1]"
338,welt,https://www.welt.de/politik/ausland/article242...,"krieg, sp, ukraine, ns, spd, rki, schi, ss, rn...",2022-12-10,2022-12-10,2024-10-20,NaN,NaN,"Newsteam, Leopard 2, Russland-Ukraine-Krieg (2...",Leopard 2: USA fuer Lieferung deutscher Kampfp...,Kanzler Scholz will moderne westliche Kampfpan...,Kanzler Scholz will moderne westliche Kampfpan...,0.0,0.0,0.0,1.0,2616.0,True,"[0, 0, 0, 1]"
339,welt,https://www.welt.de/politik/deutschland/articl...,"terror, ss, ns",2022-12-13,2022-12-13,2024-10-20,NaN,NaN,"Schindler-Frederik, Bundestag WELT, AfD, Razzi...",Reichsbuerger-Razzia: Ermittler finden 93 Waff...,Bei der bundesweiten Razzia gegen eine Reichsb...,Bei der bundesweiten Razzia gegen eine Reichsb...,0.0,0.0,0.0,1.0,1464.0,True,"[0, 0, 0, 1]"


extract 10% labelled datapoints for later validation

In [23]:
validation_data = labelled_data.sample(n=34, random_state=42)
remaining_data = labelled_data.drop(validation_data.index)

load all unlabelled data and merge it

In [24]:
unlabelled_folder = r"C:\github\news\news\german-news\preprocess data\data_ready_for_analysis\unlabelled"

In [25]:
unlabelled_data = merge_csv_files(unlabelled_folder)

preprocessing

create new labels for each combination of labels

In [26]:
label_mapping = {
    "Fake News": 1,
    "Extreme bias": 2,
    "clickbait": 3,
    "credible": 0
}
def map_labels(row):
    labels = [label_mapping[label] for label in label_mapping if row[label] == 1]
    return "-".join(map(str, sorted(labels))) if labels else "none"

remaining_data["label"] = remaining_data.apply(map_labels, axis=1)
remaining_data

,news_outlet,provenance,query_keywords,creation_date,last_modified,crawl_date,author_person,author_organization,news_keywords,title,description,body,Fake News,Extreme bias,clickbait,credible,body_len,has_label,label
0,bild,https://www.bild.de/politik/ausland/politik-au...,"linke, migration, csu, cdu",2022-06-15,2022-06-15,2024-10-20,Filipp Piatov und Ralf Schuler,NaN,NaN,Sie verspottete Deutsche als Kartoffeln: Aktiv...,Schon wieder sorgt eine Personalentscheidung d...,Schon wieder sorgt eine Personalentscheidung d...,1.0,1.0,1.0,0.0,3574.0,True,1-2-3
1,bild,https://www.bild.de/politik/ausland/politik-au...,israel,2023-11-10,2023-11-10,2024-10-20,"P. Tiede, A. Link und H.-J. Vehlewald",NaN,NaN,Reporter als Terror-Helfer: Die perfide Propag...,"Es ist der 7. Oktober 2023. Der Tag, der alles...","Es ist der 7. Oktober 2023. Der Tag, der alles...",1.0,1.0,0.0,0.0,5226.0,True,1-2
2,bild,https://www.bild.de/politik/inland/politik-inl...,"fdp, csu, spd, cdu, afd, annalena baerbock, ch...",2023-06-19,2023-06-19,2024-10-20,Filipp Piatov,NaN,"CDU, CSU, die Gruenen, SPD, Wuest Hendrik, Mer...",Streit in der CDU: Umfrage-Schlacht zwischen M...,In der CDU liegen die Nerven blank! Es tobt ei...,In der CDU liegen die Nerven blank! Spaetesten...,0.0,1.0,0.0,0.0,3071.0,True,2
3,bild,https://www.bild.de/politik/ausland/politik-au...,"israel, spd, olaf scholz",2023-10-30,2023-10-30,2024-10-20,NaN,BILD,"Gaza-Streifen, Gaza, Israel, Hamas, Nahost-Kon...",Shani Louk ermordet Scholz: Zeigt die ganze B...,Shani Louk wurde barbarisch ermordet. Das best...,Es ist das pure Grauen. Am 7. Oktober verschle...,0.0,0.0,0.0,1.0,2655.0,True,0
4,bild,https://www.bild.de/regional/leipzig/leipzig-n...,"abschiebung, migration, cdu",2023-11-09,2023-11-09,2024-10-20,KARL KEIM,NaN,"Tunesien, Georgien, Sachsen, asylrecht, Schust...",Sachsen: Nur jede dritte Abschiebung findet wi...,In Sachsen klappt nur jede dritte Abschiebung....,Leipzig In Sachsen wurden bisher in diesem Ja...,0.0,0.0,0.0,1.0,2447.0,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,welt,https://www.welt.de/iconist/mode/article249414...,"sp, ns, spo, schi, demo, ss, rna",2024-01-10,2024-01-10,2024-10-20,NaN,NaN,"texttospeech, Uniqlo Wachsjacke, Timberland, M...",Mode: Diese modischen Investitionen lohnen sic...,Die Temperaturen? Unterirdisch eisig! Doch in ...,Die Temperaturen? Unterirdisch eisig! Doch in ...,0.0,0.0,0.0,1.0,4587.0,True,0
336,welt,https://www.welt.de/politik/ausland/article242...,"krieg, christ, ss, sp, ukraine, ns, spo, nazi,...",2022-12-12,2022-12-12,2024-10-20,NaN,NaN,"Newsteam, Russland-Ukraine-Krieg (24.2.2022), ...",Ukraine-News ++ Ukraines Verteidigungsminister...,Laut dem ukrainischen Verteidigungsminister bi...,Laut dem ukrainischen Verteidigungsminister bi...,0.0,0.0,0.0,1.0,21564.0,True,0
337,welt,https://www.welt.de/politik/deutschland/articl...,"sp, terror, ns, spd, schi, rechtsextremismus, ss",2022-12-13,2022-12-13,2024-10-20,NaN,NaN,"Newsteam, Wehrbeauftragte, Hoegl, Eva, Reichsb...",Wehrbeauftragte fordert haerteres Vorgehen geg...,Nach der Razzia gegen ein mutmassliches Terror...,Nach der Razzia gegen ein mutmassliches Terror...,0.0,0.0,0.0,1.0,972.0,True,0
339,welt,https://www.welt.de/politik/deutschland/articl...,"terror, ss, ns",2022-12-13,2022-12-13,2024-10-20,NaN,NaN,"Schindler-Frederik, Bundestag WELT, AfD, Razzi...",Reichsbuerger-Razzia: Ermittler finden 93 Waff...,Bei der bundesweiten Razzia gegen eine Reichsb...,Bei der bundesweiten Razzia gegen eine Reichsb...,0.0,0.0,0.0,1.0,1464.0,True,0


In [27]:
unique_labels = set()
for label in remaining_data['label']:
    unique_labels.update(label.split('-'))
unique_labels = sorted(unique_labels, key=int)

# Get the total number of unique labels
total_labels = len(unique_labels)

print("Unique Labels:", unique_labels)
print("Total Number of Unique Labels:", total_labels)

Unique Labels: ['0', '1', '2', '3']
Total Number of Unique Labels: 4


In [28]:
le = LabelEncoder()
remaining_data['label']= le.fit_transform(remaining_data['label']) 
remaining_data

,news_outlet,provenance,query_keywords,creation_date,last_modified,crawl_date,author_person,author_organization,news_keywords,title,description,body,Fake News,Extreme bias,clickbait,credible,body_len,has_label,label
0,bild,https://www.bild.de/politik/ausland/politik-au...,"linke, migration, csu, cdu",2022-06-15,2022-06-15,2024-10-20,Filipp Piatov und Ralf Schuler,NaN,NaN,Sie verspottete Deutsche als Kartoffeln: Aktiv...,Schon wieder sorgt eine Personalentscheidung d...,Schon wieder sorgt eine Personalentscheidung d...,1.0,1.0,1.0,0.0,3574.0,True,7
1,bild,https://www.bild.de/politik/ausland/politik-au...,israel,2023-11-10,2023-11-10,2024-10-20,"P. Tiede, A. Link und H.-J. Vehlewald",NaN,NaN,Reporter als Terror-Helfer: Die perfide Propag...,"Es ist der 7. Oktober 2023. Der Tag, der alles...","Es ist der 7. Oktober 2023. Der Tag, der alles...",1.0,1.0,0.0,0.0,5226.0,True,6
2,bild,https://www.bild.de/politik/inland/politik-inl...,"fdp, csu, spd, cdu, afd, annalena baerbock, ch...",2023-06-19,2023-06-19,2024-10-20,Filipp Piatov,NaN,"CDU, CSU, die Gruenen, SPD, Wuest Hendrik, Mer...",Streit in der CDU: Umfrage-Schlacht zwischen M...,In der CDU liegen die Nerven blank! Es tobt ei...,In der CDU liegen die Nerven blank! Spaetesten...,0.0,1.0,0.0,0.0,3071.0,True,9
3,bild,https://www.bild.de/politik/ausland/politik-au...,"israel, spd, olaf scholz",2023-10-30,2023-10-30,2024-10-20,NaN,BILD,"Gaza-Streifen, Gaza, Israel, Hamas, Nahost-Kon...",Shani Louk ermordet Scholz: Zeigt die ganze B...,Shani Louk wurde barbarisch ermordet. Das best...,Es ist das pure Grauen. Am 7. Oktober verschle...,0.0,0.0,0.0,1.0,2655.0,True,0
4,bild,https://www.bild.de/regional/leipzig/leipzig-n...,"abschiebung, migration, cdu",2023-11-09,2023-11-09,2024-10-20,KARL KEIM,NaN,"Tunesien, Georgien, Sachsen, asylrecht, Schust...",Sachsen: Nur jede dritte Abschiebung findet wi...,In Sachsen klappt nur jede dritte Abschiebung....,Leipzig In Sachsen wurden bisher in diesem Ja...,0.0,0.0,0.0,1.0,2447.0,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,welt,https://www.welt.de/iconist/mode/article249414...,"sp, ns, spo, schi, demo, ss, rna",2024-01-10,2024-01-10,2024-10-20,NaN,NaN,"texttospeech, Uniqlo Wachsjacke, Timberland, M...",Mode: Diese modischen Investitionen lohnen sic...,Die Temperaturen? Unterirdisch eisig! Doch in ...,Die Temperaturen? Unterirdisch eisig! Doch in ...,0.0,0.0,0.0,1.0,4587.0,True,0
336,welt,https://www.welt.de/politik/ausland/article242...,"krieg, christ, ss, sp, ukraine, ns, spo, nazi,...",2022-12-12,2022-12-12,2024-10-20,NaN,NaN,"Newsteam, Russland-Ukraine-Krieg (24.2.2022), ...",Ukraine-News ++ Ukraines Verteidigungsminister...,Laut dem ukrainischen Verteidigungsminister bi...,Laut dem ukrainischen Verteidigungsminister bi...,0.0,0.0,0.0,1.0,21564.0,True,0
337,welt,https://www.welt.de/politik/deutschland/articl...,"sp, terror, ns, spd, schi, rechtsextremismus, ss",2022-12-13,2022-12-13,2024-10-20,NaN,NaN,"Newsteam, Wehrbeauftragte, Hoegl, Eva, Reichsb...",Wehrbeauftragte fordert haerteres Vorgehen geg...,Nach der Razzia gegen ein mutmassliches Terror...,Nach der Razzia gegen ein mutmassliches Terror...,0.0,0.0,0.0,1.0,972.0,True,0
339,welt,https://www.welt.de/politik/deutschland/articl...,"terror, ss, ns",2022-12-13,2022-12-13,2024-10-20,NaN,NaN,"Schindler-Frederik, Bundestag WELT, AfD, Razzi...",Reichsbuerger-Razzia: Ermittler finden 93 Waff...,Bei der bundesweiten Razzia gegen eine Reichsb...,Bei der bundesweiten Razzia gegen eine Reichsb...,0.0,0.0,0.0,1.0,1464.0,True,0


In [29]:
remaining_data.dtypes

news_outlet             object
provenance              object
query_keywords          object
creation_date           object
last_modified           object
crawl_date              object
author_person           object
author_organization     object
news_keywords           object
title                   object
description             object
body                    object
Fake News              float64
Extreme bias           float64
clickbait              float64
credible               float64
body_len               float64
has_label                 bool
label                    int64
dtype: object

In [30]:
unique_labels = set()
for label in remaining_data['label']:
    # Convert integer to string, then split into individual digits
    unique_labels.update(str(label))

# Convert the set to a sorted list
unique_labels = sorted(unique_labels, key=int)
print("Unique Labels:", unique_labels)

Unique Labels: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [31]:

#train_data, test_data = train_test_split(remaining_data, test_size=0.2, random_state=42)

#TARGET_COLUMN = ["Fake News", "Extreme bias", "clickbait", "credible"]
TARGET_COLUMN = "label"
FEATURE_COLUMNS = ["title",	"description",	"body"]

X = remaining_data[['title', 'description', 'body']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
y = remaining_data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)  # Should match len(y_train)
print(y_train.shape) 
print(len(y_train))
print(X_train.shape)

(245,)
(245,)
245
(245,)


bag of words

In [32]:
count_vectorizer = CountVectorizer()
X_train = count_vectorizer.fit_transform(X_train)
X_test = count_vectorizer.transform(X_test)

In [33]:
X_train.shape #(3,3)
#y_train.shape #(222,)

(245, 21482)

In [ ]:
def compute_class_weights(y):
    classes = np.unique(y)
    weights = class_weight.compute_class_weight('balanced', classes=classes, y=y)
    return dict(zip(classes, weights))

class_weights = compute_class_weights(y_train)

naive bayes model

In [35]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
nb_predictions = nb_model.predict(X_test)
print("Naive Bayes Classification Report:")
print(classification_report(y_test, nb_predictions))

Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       0.60      1.00      0.75        37
           2       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         9
          10       0.00      0.00      0.00         7
          11       0.00      0.00      0.00         2

    accuracy                           0.60        62
   macro avg       0.09      0.14      0.11        62
weighted avg       0.36      0.60      0.45        62



c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [36]:
pred = nb_model.predict(X_test)

decision tree model

In [37]:
dt_model = DecisionTreeClassifier(random_state=42, class_weight=class_weights)
dt_model.fit(X_train, y_train)
dt_predictions = dt_model.predict(X_test)
print("Decision Tree Classification Report:")
print(classification_report(y_test, dt_predictions))

Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.62      0.61        37
           2       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         3
           7       0.25      0.33      0.29         9
          10       0.00      0.00      0.00         7
          11       0.00      0.00      0.00         2

    accuracy                           0.42        62
   macro avg       0.10      0.12      0.11        62
weighted avg       0.39      0.42      0.40        62



c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

random forest

In [38]:
rf_model = RandomForestClassifier(random_state=42, class_weight=class_weights)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
print("Random Forest Classification Report:")
print(classification_report(y_test, rf_predictions))

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.60      1.00      0.75        37
           2       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         9
          10       0.00      0.00      0.00         7
          11       0.00      0.00      0.00         2

    accuracy                           0.60        62
   macro avg       0.09      0.14      0.11        62
weighted avg       0.36      0.60      0.45        62



c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

evaluation

In [39]:
X_validation = count_vectorizer.transform(validation_data)
y_validation = validation_data[TARGET_COLUMN].astype("category").cat.codes
validation_predictions = rf_model.predict(X_validation)
print("Validation Data Classification Report:")
print(classification_report(y_validation, validation_predictions, target_names=y_train.astype("category").cat.categories))


Validation Data Classification Report:


ValueError: Found input variables with inconsistent numbers of samples: [34, 19]